In [18]:
import gensim
import pickle
import pandas as pd

from gensim.models.coherencemodel import CoherenceModel

In [23]:
df = pd.read_csv("../../df_lemmatize.csv")

In [24]:
df.columns

Index(['Unnamed: 0', 'journal_name', 'year', 'title', 'author', 'author_info',
       'abstract', 'lemmatize_abstract'],
      dtype='object')

In [25]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [26]:
import ast
def to_list(x):
    return ast.literal_eval(x)

df['lemmatize_abstract'] = df.lemmatize_abstract.map(to_list)

In [27]:
from gensim import corpora

dictionary = corpora.Dictionary(df.lemmatize_abstract)
corpus = [dictionary.doc2bow(text) for text in df.lemmatize_abstract]

In [31]:
corpus[10000]

[(10, 2),
 (21, 1),
 (58, 1),
 (60, 1),
 (70, 1),
 (76, 2),
 (83, 1),
 (89, 1),
 (99, 2),
 (116, 1),
 (149, 2),
 (156, 1),
 (162, 1),
 (165, 1),
 (176, 3),
 (188, 1),
 (199, 1),
 (219, 5),
 (227, 4),
 (256, 4),
 (271, 1),
 (313, 1),
 (343, 1),
 (350, 1),
 (354, 1),
 (378, 1),
 (383, 3),
 (387, 1),
 (388, 1),
 (389, 1),
 (405, 1),
 (409, 1),
 (458, 1),
 (486, 1),
 (501, 1),
 (504, 1),
 (551, 2),
 (554, 2),
 (559, 2),
 (586, 1),
 (599, 2),
 (611, 6),
 (615, 2),
 (637, 1),
 (664, 5),
 (671, 7),
 (677, 2),
 (726, 1),
 (757, 2),
 (758, 1),
 (815, 1),
 (944, 2),
 (982, 1),
 (1075, 2),
 (1083, 5),
 (1084, 1),
 (1093, 2),
 (1098, 1),
 (1105, 1),
 (1122, 1),
 (1144, 1),
 (1167, 1),
 (1173, 1),
 (1174, 1),
 (1332, 1),
 (1337, 3),
 (1403, 1),
 (1467, 1),
 (1744, 1),
 (1794, 1),
 (1966, 3),
 (2142, 1),
 (2158, 5),
 (2165, 1),
 (2166, 1),
 (2290, 1),
 (2322, 1),
 (2384, 1),
 (2450, 3),
 (2872, 8),
 (3120, 1),
 (3434, 1),
 (3588, 1),
 (3659, 1),
 (3807, 2),
 (4976, 1),
 (5169, 1),
 (5500, 6),
 (6091

In [7]:
import gensim

# k = 4
# model = gensim.models.ldamodel.LdaModel(corpus, # input data
#                                         num_topics=k, # 토픽 모델링을 하고싶은 갯수
#                                         id2word=dictionary, 
#                                         passes=5,
#                                         eval_every=None)

# model.save("../files/lda_topic4.model")

In [11]:
from gensim.models.ldamodel import LdaModel

model = LdaModel.load("../files/lda_topic4.model")

In [32]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

vis = gensimvis.prepare(model, corpus, dictionary)

/Users/geoffrey/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/Users/geoffrey/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecati

In [33]:
pyLDAvis.display(vis)

In [34]:
from tqdm.notebook import tqdm

# 토픽1의 비율이 50%가 넘는 문서의 index
topic_1_list = []
check_topic = model[corpus]

for i, topic_dist in tqdm(enumerate(check_topic)):
    for idx, dist in topic_dist:
        if idx == 1 and dist >= 0.5:
            topic_1_list.append(i)

In [35]:

from operator import itemgetter

# 가장 dist가 높은 토픽 & 분포 저장
main_topic = []
for topic_list in tqdm(model[corpus]):
    main_topic.append(max(topic_list, key=itemgetter(1)))
    

In [36]:
model[corpus][0]

[(0, 0.3006145), (2, 0.528884), (3, 0.16877368)]

In [37]:
main_topic[0]

(2, 0.5288837)

In [38]:
# 특정 토픽이 60% 넘을 경우 해당 토픽으로 indexing

def get_over_60(x):
    if x[1] >= 0.6:
        return x[0]
    else:
        return -1
    # 참고) 같은 표현
    # return x[0] if x[1] >= 0.6 else -1


topics = list(map(get_over_60, main_topic))

In [39]:
df['topic'] = topics

In [54]:
df[df.topic == 2][['abstract', 'topic']].iloc[[0,1,2]]

,abstract,topic
2,Polymeric material plays an important role as ...,2
4,Recent studies have reported a relationship be...,2
7,P. gingivalis (Pg) is an oral anaerobe which d...,2


In [35]:
test_df = df[df.topic != -1]

In [41]:
test_df_1980 = test_df[(test_df.year >= 1980) & (test_df.year < 1990)]
def topic_percent(x):
    return x/sum(test_df_1980.topic.value_counts())

result_df_1980 = test_df_1980.topic.value_counts().map(topic_percent)
result_df_1980 = result_df_1980.sort_index()
result_df_1980

0    0.460227
1    0.164773
2    0.202652
3    0.172348
Name: topic, dtype: float64

In [39]:
test_df_1990 = test_df[(test_df.year >= 1990) & (test_df.year < 2000)]
result_df_1990 = test_df_1990.topic.value_counts() / sum(test_df_1990.topic.value_counts())
result_df_1990 = result_df_1990.sort_index()
result_df_1990




0    0.385229
1    0.207090
2    0.191433
3    0.216248
Name: topic, dtype: float64

In [42]:
test_df_2000 = test_df[(test_df.year >= 2000) & (test_df.year < 2010)]
result_df_2000 = test_df_2000.topic.value_counts() / sum(test_df_2000.topic.value_counts())
result_df_2000 = result_df_2000.sort_index()
result_df_2000

0    0.337291
1    0.224746
2    0.227327
3    0.210635
Name: topic, dtype: float64

In [43]:
test_df_2010 = test_df[(test_df.year >= 2010) & (test_df.year < 2022)]
result_df_2010 = test_df_2010.topic.value_counts() / sum(test_df_2010.topic.value_counts())
result_df_2010 = result_df_2010.sort_index()
result_df_2010

0    0.253036
1    0.279040
2    0.291185
3    0.176738
Name: topic, dtype: float64

In [44]:
result_df = pd.concat([result_df_1980, result_df_1990, result_df_2000, result_df_2010], axis=1)
result_df.columns = ['topic_1980', 'topic_1990', 'topic_2000', 'topic_2010']
result_df

,topic_1980,topic_1990,topic_2000,topic_2010
0,0.460227,0.385229,0.337291,0.253036
1,0.164773,0.207090,0.224746,0.279040
2,0.202652,0.191433,0.227327,0.291185
3,0.172348,0.216248,0.210635,0.176738
